In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle

import pandas as pd

mentions = pickle.load(open("../data/mentions texts.pickle", "rb"))
mentions.drop(columns=["messageid"], inplace=True)
mentions.drop(index=mentions[mentions.MessageText.str.len() == 0].index, inplace=True)
mentions.drop(index=mentions[mentions.issuerid < 0].index, inplace=True)
sentiment = pickle.load(open("../data/sentiment_texts.pickle", "rb"))
sentiment.drop(
    index=sentiment[sentiment.MessageText.str.len() == 0].index, inplace=True
)
print(len(mentions), len(sentiment))

ment = mentions[["issuerid", "MessageText"]]
sent = sentiment[["issuerid", "MessageText", "SentimentScore"]]

mrg = pd.merge(ment, sent, on=["issuerid", "MessageText"], how="right")

In [ ]:
def apply_dataset_simple(group: pd.DataFrame):
    return ";".join(map(str, group["issuerid"].tolist()))


def apply_dataset_hard(group: pd.DataFrame):
    issuer = group["issuerid"].tolist()
    score = group["SentimentScore"].tolist()
    return ";".join([f"{i}-{s}" for i, s in zip(issuer, score)])


messages = []
labels = []
for message, group in mentions.groupby("MessageText"):
    messages.append(message)
    labels.append(apply_dataset_simple(group))

dataset = pd.DataFrame({"message": messages, "label": labels})
dataset.to_csv("../data/data.csv", index=False)

In [ ]:
messages = []
labels = []
for message, group in mrg.groupby("MessageText"):
    messages.append(message)
    labels.append(apply_dataset_hard(group))

dataset = pd.DataFrame({"message": messages, "label": labels})
dataset.to_csv("../data/data-hard.csv", index=False)